To run the array ephys workflow, we need to properly set up the DataJoint configuration. The configuration will be saved in a file called `dj_local_conf.json` on each machine and this notebook walks you through the process.


**The configuration only needs to be set up once**, if you have gone through the configuration before, directly go to [02-workflow-structure](02-workflow-structure-optional.ipynb).

# Set up configuration in root directory of this package

As a convention, we set the configuration up in the root directory of the workflow package and always starts importing datajoint and pipeline modules from there.

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()
import os
import getpass
import datajoint as dj
import u19_pipeline.utility as utility

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


# Configure database host address and credentials

Now let's set up the host, user and password in the `dj.config` global variable

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = 'alvaros'
dj.config['database.password'] = getpass.getpass('Enter your password:') # enter the password securily

You should be able to connect to the database at this stage.

In [3]:
dj.conn()

Connecting alvaros@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) alvaros@datajoint00.pni.princeton.edu:3306

#### if Windows:
+ install Graphviz for diagram drawing.
+ Add Graphviz to path

In [4]:
if os.name == 'nt':
    if 'C:\\Program Files\\Graphviz\\bin' not in os.environ["PATH"]:
        os.environ["PATH"] += ';C:\\Program Files\\Graphviz\\bin'

# Configure the `custom` field in `dj.config` for the element-array-ephys

The major component of the current workflow is the [DataJoint Array Ephys Element](https://github.com/datajoint/element-array-ephys). Array Ephys Element requires configurations in the field `custom` in `dj.config`:

## Database prefix

Giving a prefix to schema could help on the configuration of privilege settings. For example, if we set prefix `u19_`, every schema created with the current workflow will start with `u19_`.

The prefix could be configurated as follows in `dj.config`:

In [5]:
if 'custom' not in dj.config:
    dj.config['custom'] = dict()

In [6]:
dj.config['custom'] = {'database.prefix': 'u19_'}

We also have to define the external storage location. This is the remote directory on bucket, `smb://bucket.pni.princeton.edu/u19_dj` where datajoint stores bulky data. This location should be mounted locally

In [7]:
location_extstorage = utility.get_network_path('u19_dj') + '/external_dj_blobs'
dj.config['stores'] = {
'extstorage':
{
    'location': location_extstorage,
    'protocol': 'file'
}
}

## Root directories for raw ephys data and kilosort 2 processed results

+ `ephys_root_data_dir` field indicates the root directory for the **ephys raw data** from SpikeGLX or OpenEphys (e.g. `*imec0.ap.bin`, `*imec0.ap.meta`, `*imec0.lf.bin`, `imec0.lf.meta`) or the **clustering results** from kilosort2 (e.g. `spike_times.npy`, `spike_clusters.npy`). The root path typically **do not** contain information of subjects or sessions, all data from subjects/sessions should be subdirectories in the root path.

+ In the database, every path for the ephys raw data is **relative to this root path**. The benefit is that the absolute path could be configured for each machine, and when data transfer happens, we just need to change the root directory in the config file.
+ The workflow supports **multiple root directories**. If there are multiple possible root directories, specify the `ephys_root_data_dir` as a list.
+ The root path(s) are **specific to each machine**, as the name of drive mount could be different for different operating systems or machines.
+ In the context of the workflow, all the paths saved into the database or saved in the config file need to be in the **POSIX standards** (Unix/Linux), with `/`. The path conversion for machines of any operating system is taken care of inside the elements.

### Examples
+ U19 mesoscope directory for Linux systems: `/mnt/Bezos-center/Manuel/neuropixel/manuel`
+ U19 mesoscope directory for Mac systems:   `/Volumes/Bezos-center/Manuel/neuropixel/manuel`
+ U19 mesoscope directory for Windows systems: `//bucket.pni.princeton.edu/Bezos-center/Manuel/neuropixel/manuel`

In [8]:
# If there is only one root path.
dj.config['custom']['ephys_root_data_dir'] = utility.get_network_path('Bezos') + '/Manuel/neuropixel/manuel'
# If there are multiple possible root paths:
#dj.config['custom']['ephys_root_data_dir'] = ['/mnt/bucket/labs/tank']

In [9]:
dj.config['custom']

{'database.prefix': 'u19_',
 'ephys_root_data_dir': '//bucket.pni.princeton.edu/Bezos-center/Manuel/neuropixel/manuel'}

# Save the configuration as a json file

With the proper configurations, we could save this as a file, either as a local json file, or a global file.

In [10]:
dj.config.save_local()

Local configuration file is saved as `dj_local_conf.json` in the root directory of this package `U19-pipeline_python`. Next time if you change your directory to `U19-pipeline_python` before importing datajoint and the pipeline packages, the configurations will get properly loaded.

If saved globally, there will be a hidden configuration file saved in your root directory. The configuration will be loaded no matter where the directory is.

In [ ]:
# dj.config.save_global()

# Next Step

After the configuration, we will be able to review the ephys related schemas and tables structure with [01-ephys-workflow-structure](01-ephys-workflow-structure.ipynb).